In [1]:
import os
os.chdir('../')

In [2]:
import tensorflow as tf
import os, re
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

import tf_extend as tfe

from nets.cascadeOut import CascadeOut
net = CascadeOut()

wider_anno_file = '/home/luojiapeng/datasets/widerface/wider_face_split/wider_face_val_bbx_gt.txt'
wider_base_dir = '/home/luojiapeng/datasets/widerface/WIDER_val/images'

output_dir = tfe.get_checkpoint_dir('result', 't3.5_3')
model_dir = 'checkpoints/mse_t35_0_bak'
params = {'thres': 0.5}

def get_wider_imgs():
    anno_file = open(wider_anno_file, 'r')
    fnames = []
    for line in anno_file.readlines():
        line = line.rstrip()
        if re.match('^.*\.jpg$', line):
            fnames.append(line)
    return fnames


def wider_eval_input_fn():
    fnames = get_wider_imgs()
    fnames = [os.path.join(wider_base_dir, x) for x in fnames]
    dataset = tf.data.Dataset.from_tensor_slices((fnames,))

    def fn(fname):
        raw_img = tf.read_file(fname)
        image = tf.image.decode_jpeg(raw_img, channels=3)
        image = tf.cast(image, tf.float32)
        return {'fname': fname, 'image': image}

    dataset = dataset.map(fn, num_parallel_calls=8)
    return dataset

def save_wider_result(output_dir, fnames, results):
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    current_event = ''
    i=-1
    for res in results:
        i+=1
        fname = fnames[i]
        bboxes, scores = res['bboxes'], res['scores']
        assert len(bboxes) == len(scores)
        event = fname.split('/')[0]
        if current_event != event:
            current_event = event
            save_path = os.path.join(output_dir, current_event)
            if not os.path.exists(save_path):
                os.mkdir(save_path)
            print('current path:', current_event)

        out_fname = fname.split('.jpg')[0]
        out_fname = os.path.join(output_dir, out_fname + '.txt')
        fid = open(out_fname, 'w')
        fid.write(fname.split('/')[-1] + '\n')
        if bboxes is None:
            fid.write(str(1) + '\n')
            fid.write('%f %f %f %f %f\n' % (0, 0, 0, 0, 0.01))
            continue
        else:
            fid.write(str(len(bboxes)) + '\n')
            for _i in range(len(scores)):
                s, b =scores[_i], bboxes[_i]
                fid.write('%.2f %.2f %.2f %.2f %.2f\n' % (b[1], b[0], b[3] - b[1] + 1, b[2] - b[0] + 1, s))

            fid.close()
            if i % 10 == 0 and i:
                print(i)
    if i+1 != len(fnames):
        raise Exception('length of fnames is not consistent with results')

def eval_on_wider():
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    runConfig = tf.estimator.RunConfig(model_dir=model_dir)
    classifier = tf.estimator.Estimator(model_fn=net.model_fn,
                                        params=params,
                                        config=runConfig)
    results = classifier.predict(wider_eval_input_fn)
    save_wider_result(output_dir, get_wider_imgs(), results)


if __name__=='__main__':
    tf.logging.set_verbosity(tf.logging.DEBUG)
    eval_on_wider()


INFO:tensorflow:get_checkpoint_dir: result/t3.5_3_3
INFO:tensorflow:Using config: {'_model_dir': 'checkpoints/mse_t35_0_bak', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fef6f88f4a8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from checkpoints/mse_t35_0_bak/model.ckpt-185000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
current path: 0--P